In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import json

url = 'https://www.baraasallout.com/test.html?authuser=1'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

def extract_text_data():
    headings = []
    paragraphs = []
    list_items = []

    for heading in soup.find_all(['h1', 'h2']):
        headings.append(heading.get_text())

    for p in soup.find_all('p'):
        paragraphs.append(p.get_text())

    for li in soup.find_all('li'):
        list_items.append(li.get_text())

    with open('Extract_Text_Data.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Headings', 'Paragraphs', 'List Items'])
        for i in range(max(len(headings), len(paragraphs), len(list_items))):
            writer.writerow([headings[i] if i < len(headings) else '',
                             paragraphs[i] if i < len(paragraphs) else '',
                             list_items[i] if i < len(list_items) else ''])
def extract_table_data():
    table_data = []

    table = soup.find('table')
    if table:
        rows = table.find_all('tr')
        for row in rows[1:]: 
            cols = row.find_all('td')
            if len(cols) > 2: 
                product_name = cols[0].get_text()
                price = cols[1].get_text()
                stock_status = cols[2].get_text()
                table_data.append([product_name, price, stock_status])

        with open('Extract_Table_Data.csv', 'w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['Product Name', 'Price', 'Stock Status'])
            writer.writerows(table_data)

def extract_product_info():
    product_data = []

    cards = soup.find_all('div', class_='product-card') 
    for card in cards:
        title = card.find('h3').get_text()
        price = card.find('span', class_='price').get_text() 
        stock = card.find('span', class_='stock').get_text() if card.find('span', class_='stock') else 'N/A' 
        button_text = card.find('button').get_text() if card.find('button') else 'No button' 
        product_data.append({
            'title': title,
            'price': price,
            'stock': stock,
            'button_text': button_text
        })
    with open('Product_Information.json', 'w', encoding='utf-8') as file:
        json.dump(product_data, file, ensure_ascii=False, indent=4)
def extract_form_details():
    form_data = []

    forms = soup.find_all('form')
    for form in forms:
        fields = form.find_all(['input', 'textarea', 'select'])
        for field in fields:
            field_name = field.get('name')
            input_type = field.get('type')
            default_value = field.get('value', 'No default value')
            form_data.append({
                'field_name': field_name,
                'input_type': input_type,
                'default_value': default_value
            })
    with open('Form_Details.json', 'w', encoding='utf-8') as file:
        json.dump(form_data, file, ensure_ascii=False, indent=4)

def extract_links_and_multimedia():
    link_data = []

    links = soup.find_all('a', href=True)
    for link in links:
        link_data.append({'url': link['href']})

    iframes = soup.find_all('iframe', src=True)
    for iframe in iframes:
        link_data.append({'video_url': iframe['src']})

    with open('Links_and_Multimedia.json', 'w', encoding='utf-8') as file:
        json.dump(link_data, file, ensure_ascii=False, indent=4)

def extract_featured_products():
    featured_products = []

    products = soup.find_all('span', class_='name')
    for product in products:
        name = product.get_text()
        price = product.find_next('span', class_='price').get_text() 
        colors = product.find_next('span', class_='colors').get_text() 
        product_id = product.get('data-id') 
        featured_products.append({
            'id': product_id,
            'name': name,
            'price': price,
            'colors': colors
        })
    with open('Featured_Products.json', 'w', encoding='utf-8') as file:
        json.dump(featured_products, file, ensure_ascii=False, indent=4)

extract_text_data()
extract_table_data()
extract_product_info()
extract_form_details()
extract_links_and_multimedia()
extract_featured_products()
